<a href="https://colab.research.google.com/github/debrupf2946/Personal_projects/blob/main/cifar_100_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d fedesoriano/cifar100

 94% 151M/161M [00:01<00:00, 138MB/s]
100% 161M/161M [00:01<00:00, 116MB/s]


In [3]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,MaxPool2D,Conv2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

In [4]:
import zipfile
zip_ref=zipfile.ZipFile("/content/cifar100.zip","r")
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

In [6]:
trainData = unpickle('train')
#type of items in each file
for item in trainData:
    print(item, type(trainData[item]))


filenames <class 'list'>
batch_label <class 'str'>
fine_labels <class 'list'>
coarse_labels <class 'list'>
data <class 'numpy.ndarray'>


In [7]:
testData= unpickle('test')
for items in testData:
  print(item,type(testData[item]))

data <class 'numpy.ndarray'>
data <class 'numpy.ndarray'>
data <class 'numpy.ndarray'>
data <class 'numpy.ndarray'>
data <class 'numpy.ndarray'>


In [8]:
X_train=trainData['data']

In [9]:
X_train=X_train.reshape(len(X_train),3,32,32).transpose(0,2,3,1)
X_train.shape

(50000, 32, 32, 3)

In [10]:
X_test = testData['data']
X_test = X_test.reshape(len(X_test),3,32,32).transpose(0,2,3,1)
X_test.shape

(10000, 32, 32, 3)

In [11]:
y_test=testData["fine_labels"]
y_test=np.array(y_test)
y_test[:5]

array([49, 33, 72, 51, 71])

In [12]:
y_train=trainData['fine_labels']
X_train=np.array(X_train)
y_train=np.array(y_train)

In [13]:
y_train.shape

(50000,)

In [14]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,train_size=0.8)
X_valid=np.array(X_valid)
y_valid=np.array(y_valid)

In [15]:
aug=ImageDataGenerator(rescale=1.0/255.,
                             rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1,
                             zoom_range=0.2,
                             fill_mode='nearest',
                             horizontal_flip=True)
resacler=ImageDataGenerator(rescale=1.0/255.)

In [16]:
train_generator=aug.flow(X_train,y_train,batch_size=40)
valid_generator=resacler.flow(X_valid,y_valid, batch_size=40)
test_genrator=resacler.flow(X_test,y_test, batch_size=40)

In [17]:
model=Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding="valid",activation="relu",input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding="valid"))

model.add(Conv2D(64,kernel_size=(3,3),padding="valid",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding="valid"))

model.add(Conv2D(128,kernel_size=(3,3),padding="valid",activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding="valid"))

model.add(Flatten())

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(128,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(100,activation="softmax"))

In [18]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [19]:
X_train=np.array(X_train)
y_train=np.array(y_train)
X_valid=np.array(X_valid)
y_valid=np.array(y_valid)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', patience=5, min_delta=0.002)


In [21]:
history=model.fit(train_generator,validation_data=valid_generator,batch_size=40,epochs=100,callbacks=early_stopping)

Epoch 1/100
1000/1000 [==============================] - 46s 31ms/step - loss: 3.7915 - accuracy: 0.1272 - val_loss: 3.6293 - val_accuracy: 0.1527
Epoch 2/100
1000/1000 [==============================] - 31s 31ms/step - loss: 3.2460 - accuracy: 0.2098 - val_loss: 3.2098 - val_accuracy: 0.2222
Epoch 3/100
1000/1000 [==============================] - 31s 31ms/step - loss: 2.9899 - accuracy: 0.2564 - val_loss: 2.7512 - val_accuracy: 0.3022
Epoch 4/100
1000/1000 [==============================] - 31s 31ms/step - loss: 2.8155 - accuracy: 0.2881 - val_loss: 2.9851 - val_accuracy: 0.2685
Epoch 5/100
1000/1000 [==============================] - 32s 32ms/step - loss: 2.6853 - accuracy: 0.3179 - val_loss: 2.5432 - val_accuracy: 0.3482
Epoch 6/100
1000/1000 [==============================] - 31s 31ms/step - loss: 2.5833 - accuracy: 0.3357 - val_loss: 2.8853 - val_accuracy: 0.2979
Epoch 7/100
1000/1000 [==============================] - 32s 32ms/step - loss: 2.4824 - accuracy: 0.3566 - val_loss: 2